##1번 문제

- 개요
 - BNPL은 후불결제 서비스이며, 연초에 도입되었고 많은 논란이 있습니다. BNPL에 대한 온라인 뉴스를 2020년 3월부터 수집해서 분석하고 토픽을 발견하고 해석해보세요. 토픽별로 중요한 단어를 발견하시고, W2V를 활용하여 유사한 단어들도 찾아보세요.

- 데이터
 - 온라인 포털 내 검색된 뉴스
 - 검색어: 후불결제

1. 수집대상 뉴스 검색 및 제목 저장

In [ ]:
from bs4 import BeautifulSoup
from lxml import etree
from urllib import parse
import requests
import urllib.request
import urllib.parse
import numpy as np
import pandas as pd
import time

#----------------------------------------------#
#변경할 부분
startdate = "2020.03.01"
enddate = "2022.10.31"
searchword ="후불결제"#검색어
#----------------------------------------------#

base_url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query"+parse.urlencode([('',searchword)], doseq=True)+"&sort=1&photo=3&field=0&pd=3&ds="+startdate+"&de="+enddate+"&mynews=1&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from"+startdate.replace(".", "")+"to"+enddate.replace(".", "")+",a:all&mynews=0&start="
titles = [ ]
urls = [ ]
i=1

while True:
 try:
    time.sleep(0.45)
    url = base_url+str(i)
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    t_news = soup.select('a.news_tit')  #클래스 표시
    t_len = len(t_news)

    if t_len <1:
        break

    for j in np.arange(t_len)+1:
            try: 
                titles.append(soup.select('a.news_tit')[j].text)
                urls.append(soup.select('a.info')[j]['href'])
            except Exception as e:
                continue
    i=i+10
    
 except Exception as e:
    continue

result = pd.DataFrame({"Url": urls, "Title":titles})
result.Url[0]

'https://n.news.naver.com/mnews/article/016/0002055559?sid=101'

In [ ]:
result.shape

(558, 2)

2. 본문 수집

In [ ]:
articles = []
dates = []

for i in result.Url: 
   try:
    if( "news.naver.com" in i ): #바뀐곳
     print(i)
     time.sleep(0.45)
     html = urllib.request.urlopen(i).read()
     soup = BeautifulSoup(html, 'html.parser')

     news = soup.select('div#dic_area')[0].text #id #, class.
     news_date = soup.select('span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME')[0].text  #네이버 뉴스의 날짜 시간 부분 지정

     articles.append(news)
     dates.append(news_date.split()[0])

   except Exception as e:
    print(e)
    continue


https://n.news.naver.com/mnews/article/016/0002055559?sid=101
https://n.news.naver.com/mnews/article/005/0001560655?sid=101
https://n.news.naver.com/mnews/article/029/0002760524?sid=101
https://n.news.naver.com/mnews/article/030/0003052370?sid=101
https://n.news.naver.com/mnews/article/011/0004110681?sid=101
https://n.news.naver.com/mnews/article/005/0001554453?sid=101
https://n.news.naver.com/mnews/article/015/0004752585?sid=101
https://n.news.naver.com/mnews/article/015/0004751518?sid=101
https://n.news.naver.com/mnews/article/008/0004793537?sid=101
https://n.news.naver.com/mnews/article/029/0002752961?sid=101
https://n.news.naver.com/mnews/article/011/0004090667?sid=104
https://n.news.naver.com/mnews/article/009/0005005231?sid=101
https://n.news.naver.com/mnews/article/021/0002526605?sid=101
https://n.news.naver.com/mnews/article/277/0005132644?sid=101
https://n.news.naver.com/mnews/article/025/0003216208?sid=103
https://n.news.naver.com/mnews/article/029/0002744745?sid=101
https://

In [ ]:
# 수집된 내용을 표로 정리하기, news컬럼, date컬럼, likes컬럼
df = pd.DataFrame({'news':articles, 'date':dates})
df

news         date
0    \n‘어둠의 짠테크’ 대리구매 실태중고거래·재테크 카페에 글 게시카드·BNPL 결제...  2022.10.21.
1    \n\n\n\n\n올해 말부터 ‘마이데이터’ 정보 제공 항목에 퇴직 연금과 공적 연...  2022.10.20.
2    \n\t\t\t'내 손안의 금융비서'로 불리는 마이데이터의 정보제공 범위가 크게 확...  2022.10.19.
3    \n\n\n\n\nⓒ게티이미지뱅크마이데이터(본인신용정보관리업)에서 월 단위로만 제공...  2022.10.19.
4    \n운영 5개월만에 가입자 75만명일년 넘게 운영한 네이버보다 多양사 연체율 1%대...  2022.10.16.
..                                                 ...          ...
305  \n\n\n\n\n생명보험사와 손해보험사간 양강 구도를 위협하는 새로운 도전자가 등...  2020.04.07.
306  \n\n\n\n\n전주형 재난기본소득 선불카드 발급이 확정된  첫 날 대상 시민들이...  2020.04.03.
307  \n[창간 100주년/청년 100인의 ‘두 번째 돌잡이’]<중> 틀을 깬 청년들\n...  2020.04.02.
308  \n자율주행 등 과제 20건 건의[서울경제] 전국경제인연합회가 경기침체를 신산업을 ...  2020.03.19.
309  \n“자율주행차·빅데이터 활용 등  기업 혁신성장 발판 마련해야”신종 코로나바이러스...  2020.03.19.

[310 rows x 2 columns]

3. 전처리 및 명사 추출

In [ ]:
!pip install konlpy
!pip install tqdm #진행상태를 알려주는 기능 제공

import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm

print(df.isnull().values.any()) #null값이 있는지 체크
train_data = df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

train_data['news'] = train_data['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 6.5 MB/s 
     |████████████████████████████████| 465 kB 39.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
False
False


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
#값을 저장할 비어있는 리스트를 만듬.
tokenized_data = []   

# 불용어 정의: 더 추가할 수 있음
stopwords = ['을', 'ㅋㅋ', 
             '부터', '까지' '적', '의','가','이','은', '셈', '속', '직', '친', '알', '처', '산', '론','간', '스', '기', '그','만', '분', '치', '날',
             '들','는','좀','잘','걍','과','도','를','으로',
             '자','에','와','한','하다', '점', '나']

# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Okt()

for sentence in tqdm(train_data['news']): #리뷰 각각에 대해서 아래작업 반복
    tokenized_sentence = okt.nouns(sentence)  #각 리뷰에서 명사만 뽑아서 리스트, morphs
    #위에서 명사만 추출한 결과에서, 각 명사가 stopwords인지 체크하고
    #stopwords 아닌 명사만 사용함
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

len(tokenized_data)

100%|██████████| 310/310 [00:50<00:00,  6.10it/s]


310

In [ ]:
type(tokenized_data[0])
tokenized_data[0]

['어둠',
 '테크',
 '대리',
 '구매',
 '실태',
 '중고',
 '거래',
 '재테크',
 '카페',
 '글',
 '게시',
 '카드',
 '결제',
 '통해',
 '급전',
 '조달',
 '푼',
 '의뢰인',
 '현금',
 '필요',
 '대리',
 '구매',
 '이해',
 '금융',
 '당국',
 '개인',
 '거래',
 '이유',
 '팔짱',
 '불법',
 '사금융',
 '대안',
 '우려',
 '증폭',
 '전문가',
 '부업',
 '법',
 '적용',
 '단속',
 '소고기',
 '가격',
 '카드',
 '소액',
 '결제',
 '가능',
 '전업',
 '주부',
 '씨',
 '최근',
 '인터넷',
 '카페',
 '제목',
 '글',
 '게시',
 '글',
 '만원',
 '상당',
 '소고기',
 '구매',
 '수',
 '쇼핑몰',
 '링크',
 '포함',
 '곧',
 '연락',
 '달라',
 '댓글',
 '댓글',
 '대리',
 '구매',
 '씨',
 '해당',
 '상품',
 '카드',
 '결제',
 '후',
 '배송지',
 '씨',
 '주소',
 '입력',
 '이후',
 '씨',
 '만원',
 '인',
 '만원',
 '현금',
 '씨',
 '계좌',
 '거래',
 '완료',
 '씨',
 '대리',
 '구매',
 '통해',
 '이번',
 '달',
 '약',
 '만원',
 '가량',
 '절약',
 '말',
 '비상',
 '적',
 '수',
 '실제',
 '성행',
 '대리',
 '구매',
 '사례',
 '대리',
 '구매',
 '물건',
 '사고',
 '의뢰인',
 '현금',
 '대리',
 '구매',
 '자의',
 '이해관계',
 '일종',
 '카드',
 '깡',
 '최근',
 '테크',
 '열풍',
 '대리',
 '구매',
 '재테크',
 '수단',
 '확산',
 '금융',
 '당국',
 '개인',
 '거래',
 '이유',
 '단속',
 '손',
 '전문가',
 '불법',
 '사금융',
 '합법',
 '대안',
 '수',
 '우려',


In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_data) #사전 구성
corpus = [dictionary.doc2bow(text) for text in tokenized_data]  

import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, 
                                           id2word=dictionary, passes=15)

#각 토픽의 출현확률이 높은 단어를 지정된 갯수만큼 반환
topics = ldamodel.print_topics(num_words=20)

#위 내용을 출력
for topic in topics:
    print(topic)

(0, '0.026*"네이버" + 0.018*"서비스" + 0.012*"증권" + 0.011*"결제" + 0.011*"파이" + 0.010*"금융" + 0.010*"등" + 0.008*"셜" + 0.008*"것" + 0.008*"년" + 0.008*"수" + 0.008*"투자" + 0.007*"페이" + 0.007*"대표" + 0.006*"분기" + 0.006*"월" + 0.006*"사업" + 0.005*"구매" + 0.005*"일" + 0.005*"대출"')
(1, '0.034*"카카오" + 0.020*"페이" + 0.015*"서비스" + 0.014*"금융" + 0.012*"데이터" + 0.012*"보험" + 0.012*"뱅크" + 0.011*"등" + 0.010*"수" + 0.008*"것" + 0.008*"결제" + 0.007*"제공" + 0.007*"계획" + 0.007*"대출" + 0.006*"상품" + 0.005*"월" + 0.005*"신용" + 0.005*"자동차" + 0.005*"고객" + 0.005*"후불"')
(2, '0.018*"카드" + 0.016*"금융" + 0.014*"결제" + 0.014*"규제" + 0.013*"서비스" + 0.013*"테크" + 0.012*"핀" + 0.012*"법" + 0.010*"등" + 0.010*"수" + 0.008*"것" + 0.007*"후불" + 0.007*"년" + 0.007*"업계" + 0.006*"산업" + 0.006*"소액" + 0.005*"사업" + 0.005*"업무" + 0.005*"사" + 0.005*"개정안"')
(3, '0.049*"금융" + 0.021*"규제" + 0.018*"테크" + 0.014*"은행" + 0.014*"것" + 0.014*"등" + 0.012*"플랫폼" + 0.009*"빅" + 0.009*"수" + 0.009*"핀" + 0.008*"대출" + 0.008*"빅테크" + 0.008*"서비스" + 0.006*"소비자" + 0.006*"사" + 0.006*"혁신" + 0.00

- LDA 시각화

In [ ]:
!pip install pyLDAvis

import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=04fb16c06bd8e84388c0dc3c3b7b6bb8942ee63e32a94b4bd2722b0965049c79
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=4cc44a60a6cd41c08aaaf7878af8b6d1df29593709a813a5a6a96d92827a11aa
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built pyLDAvis sklearn


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


- LDA의 토픽 개수 탐색

In [ ]:
from gensim.models.ldamodel import LdaModel 
from gensim.models.coherencemodel import CoherenceModel 

coherence_values = [] 
model_list = [] 
iters = [8, 9,10, 11, 12]

for num_topics in iters: 
     model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics) 
     model_list.append(model) 
     coherencemodel = CoherenceModel(model=model, texts=tokenized_data, dictionary=dictionary, coherence='c_v') 
     coherence_values.append(coherencemodel.get_coherence()) 

for i in range(0, len(iters)):
  print(model_list[i], coherence_values[i])  #topic num별로 coherence 가 높은 것을 찾기

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

LdaModel(num_terms=7186, num_topics=8, decay=0.5, chunksize=2000) 0.3691293045051855
LdaModel(num_terms=7186, num_topics=9, decay=0.5, chunksize=2000) 0.3669662619758169
LdaModel(num_terms=7186, num_topics=10, decay=0.5, chunksize=2000) 0.3680919467765322
LdaModel(num_terms=7186, num_topics=11, decay=0.5, chunksize=2000) 0.3743963302851199
LdaModel(num_terms=7186, num_topics=12, decay=0.5, chunksize=2000) 0.38010328891186823


- sklearn으로 DocumentTerm Matrix 생성 시 사용

In [ ]:
def tw_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = okt.nouns(text)
    return tokens_ko

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, min_df=3, max_df=0.9)
tfidf_vect.fit(train_data['news'])
tfidf_matrix_train = tfidf_vect.transform(train_data['news'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=8, random_state=0)
lda.fit(tfidf_matrix_train)

print(lda.components_.shape)
lda.components_

(8, 3094)


array([[0.12500028, 0.12500025, 0.12501941, ..., 0.1250001 , 0.12500575,
        0.12500007],
       [0.12500028, 0.12500025, 0.12500008, ..., 0.1250001 , 0.12500571,
        0.12500012],
       [0.28518958, 0.35905795, 0.95577408, ..., 0.58618382, 0.32820424,
        1.20698726],
       ...,
       [0.12500029, 0.12500026, 0.12500008, ..., 0.1250001 , 0.12500582,
        0.12500007],
       [0.12500029, 0.12500025, 0.12500008, ..., 0.1250001 , 0.1250058 ,
        0.12500007],
       [0.12539187, 0.12500024, 0.12502476, ..., 0.12500579, 0.27873579,
        0.12505484]])

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #',topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:no_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names =tfidf_vect.get_feature_names()

# Topic별 가장 연관도가 높은 word를 15개만 추출
display_topics(lda, feature_names, 15)

Topic # 0
칩 규격 엘 통운 부착 소 결실 겸용 울산 전기 드림 종전 사나 유로 문의
Topic # 1
회장 손태 승 한국판 김정태 김광수 용병 윤종규 조찬 뉴딜 비공개 불균형 시계 공존 식당
Topic # 2
금융 페이 서비스 네이버 카드 테크 것 카카오 규제 보험 핀 은행 법 만원 데이터
Topic # 3
외국인 텔레콤 상담 월드 선납 외국어 영어 중국어 일본어 약정 다문화 전화 영문 거주 근처
Topic # 4
하프 다날 하차 리스 희망 벨 독서실 승 탑승 군 감염 아이콘 스티커 선후 비콘
Topic # 5
체류 실리콘밸리 직구 지방세 국세 확 전기 츠 희망 테슬라 관순 세도 아이콘 독서실 건설
Topic # 6
직구 지방세 국세 항목 공적 특약 펫 아파트 통신비 체류 거치 자도 퇴직 취소 단기
Topic # 7
제로 어펌 주가 운전 전경련 재난 이사장 한결 페이팔 서민 애프터 팬데믹 동전 사랑 레브친


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


5. Word2Vec 활용

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, 
                 size = 10, 
                 window = 5, 
                 min_count = 5, 
                 workers = 4, sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape
model.wv.vectors[1] #w2v에서 두번째 단어의 임베딩값(벡터)

np.mean( (model.wv.vectors[2]-model.wv.vectors[1])**2 )

1.3825574

In [ ]:
model.wv.vectors[1:3]

array([[-0.10754625, -0.7065311 , -1.1618512 ,  2.1579752 ,  0.12928122,
        -1.5897924 ,  1.1278508 ,  0.662575  , -2.268717  , -2.1730897 ],
       [ 0.4405132 ,  0.43339366, -1.7442356 , -0.30517048,  0.82716334,
         0.4343606 ,  0.59919894, -0.2719836 , -2.5503895 , -2.1176696 ]],
      dtype=float32)

In [ ]:
model.wv.vocab

{'테크': <gensim.models.keyedvectors.Vocab at 0x7f8c78c09b10>,
 '대리': <gensim.models.keyedvectors.Vocab at 0x7f8c79bf8310>,
 '구매': <gensim.models.keyedvectors.Vocab at 0x7f8c79bb5750>,
 '중고': <gensim.models.keyedvectors.Vocab at 0x7f8c79bb5390>,
 '거래': <gensim.models.keyedvectors.Vocab at 0x7f8c79bb5310>,
 '재테크': <gensim.models.keyedvectors.Vocab at 0x7f8c79bd4990>,
 '카페': <gensim.models.keyedvectors.Vocab at 0x7f8c79bd4d50>,
 '글': <gensim.models.keyedvectors.Vocab at 0x7f8c79bd4c90>,
 '게시': <gensim.models.keyedvectors.Vocab at 0x7f8c79bb5210>,
 '카드': <gensim.models.keyedvectors.Vocab at 0x7f8c79bb5050>,
 '결제': <gensim.models.keyedvectors.Vocab at 0x7f8c79bd40d0>,
 '통해': <gensim.models.keyedvectors.Vocab at 0x7f8c78ca2710>,
 '조달': <gensim.models.keyedvectors.Vocab at 0x7f8c78ca2650>,
 '현금': <gensim.models.keyedvectors.Vocab at 0x7f8c78ca2450>,
 '필요': <gensim.models.keyedvectors.Vocab at 0x7f8c78ca2e50>,
 '이해': <gensim.models.keyedvectors.Vocab at 0x7f8c78ca2d10>,
 '금융': <gensim.models.ke

- 단어 간 거리 계산

In [ ]:
import numpy as np
np.sqrt(np.mean(model.wv.vectors[10]-model.wv.vectors[11])**2)

0.2560543417930603

- 유사한 단어 발견

In [ ]:
print(model.wv.most_similar("투자가"))

[('긍정', 0.9900649189949036), ('수장', 0.9885004758834839), ('고제', 0.9882495403289795), ('입점', 0.9881489276885986), ('곳도', 0.988028883934021), ('하소연', 0.9877674579620361), ('공개', 0.9870057702064514), ('주자', 0.9861631393432617), ('예상', 0.9860072135925293), ('류', 0.985741376876831)]


/usr/local/lib/python3.7/dist-packages/gensim/matutils.py:737: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if np.issubdtype(vec.dtype, np.int):


5. Doc2Vec 활용


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#각 문헌을 나타내는 리스트(단어들로 묶인) 하나하나에 대해 TaggedDocument 적용
#적용하실 때, words에다 각 뉴스를 나타내는 리스트, tags에는 각 뉴스의 아이디 입력
#아이디 입력 시, [아이디] 형태로 입력: 아이디에 해당하는 값을 리스트로 표현해서 입력
tagged_data = [    TaggedDocument(words=tokenized_data[i], tags=[i]) 
for i in range(len(tokenized_data)) ]

In [ ]:
tagged_data

[TaggedDocument(words=['어둠', '테크', '대리', '구매', '실태', '중고', '거래', '재테크', '카페', '글', '게시', '카드', '결제', '통해', '급전', '조달', '푼', '의뢰인', '현금', '필요', '대리', '구매', '이해', '금융', '당국', '개인', '거래', '이유', '팔짱', '불법', '사금융', '대안', '우려', '증폭', '전문가', '부업', '법', '적용', '단속', '소고기', '가격', '카드', '소액', '결제', '가능', '전업', '주부', '씨', '최근', '인터넷', '카페', '제목', '글', '게시', '글', '만원', '상당', '소고기', '구매', '수', '쇼핑몰', '링크', '포함', '곧', '연락', '달라', '댓글', '댓글', '대리', '구매', '씨', '해당', '상품', '카드', '결제', '후', '배송지', '씨', '주소', '입력', '이후', '씨', '만원', '인', '만원', '현금', '씨', '계좌', '거래', '완료', '씨', '대리', '구매', '통해', '이번', '달', '약', '만원', '가량', '절약', '말', '비상', '적', '수', '실제', '성행', '대리', '구매', '사례', '대리', '구매', '물건', '사고', '의뢰인', '현금', '대리', '구매', '자의', '이해관계', '일종', '카드', '깡', '최근', '테크', '열풍', '대리', '구매', '재테크', '수단', '확산', '금융', '당국', '개인', '거래', '이유', '단속', '손', '전문가', '불법', '사금융', '합법', '대안', '수', '우려', '표', '일종', '개인', '카드', '깡', '인터넷', '카페', '통해', '산발', '성행', '대리', '구매', '용법', '인터넷', '카페', '사고', '물품', '링크', '당장', '현금', '

In [ ]:
k1 = [ 1,2,3,4]
[  i*2  for i in k1 if i > 2 ]


[6, 8]

In [ ]:
tagged_data[1]

TaggedDocument(words=['올해', '말', '마이', '데이터', '정보', '제공', '항목', '퇴직', '연금', '공적', '연금', '정보', '등', '추가', '마이', '데이터', '서비스', '여러', '금융기관', '분산', '개인', '금융', '정보', '곳', '상품', '추천', '등', '서비스', '금융', '위원회', '마이', '데이터', '정보', '제공', '범위', '기존', '개', '항목', '은행', '보험', '카드', '등', '개', '항목', '일', '달', '중', '마이', '데이터', '서비스', '통해', '국세', '지방세', '관세', '납부', '역', '건강', '보험료', '납부', '역', '등', '확인', '수', '월', '계약', '피', '보험', '자도', '보험', '상품', '명과', '보험', '기간', '보장', '정보', '특약', '사항', '등', '대한', '조회', '수', '카드', '결제', '취소', '및', '후불', '교통', '카드', '아파트', '관리', '비', '통신비', '해외', '직구', '등', '무', '승인', '매입', '정보', '제공', '내년', '월', '자동', '이체', '정보', '대출', '상품', '거치', '기간', '정보', '주택', '화재', '보험', '펫', '보험', '등', '소액', '단기', '보험', '관련', '정보', '등', '마이', '데이터', '서비스', '확인', '수'], tags=[1])

In [ ]:
len(tagged_data)

310

In [ ]:
model = Doc2Vec(
    epochs=40,
    window=5,
    vector_size=5,  #얻고자 하는 컬럼의 수, 한 문서를 몇개의 값의 좌표로 표현,
    min_count=2,  #최소 단어 빈도를 2
    dm =1)  #pv-dm: 1, pv-dbow:2
  
model.build_vocab(tagged_data)

In [ ]:
model.docvecs.vectors_docs.shape

(310, 5)

In [ ]:
doc_list = '전기차 배터리 중요'.split(' ')
doc_list

['전기차', '배터리', '중요']

In [ ]:
inferred_vector = model.infer_vector(doc_list)
inferred_vector

array([-0.08916046, -0.0186717 ,  0.01023282, -0.04230481,  0.0460339 ],
      dtype=float32)

- 유사한 document 발견

In [ ]:
return_docs = model.docvecs.most_similar(positive=[inferred_vector],topn=5)
return_docs

/usr/local/lib/python3.7/dist-packages/gensim/matutils.py:737: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if np.issubdtype(vec.dtype, np.int):


[(157, 0.9537976980209351),
 (64, 0.9416919946670532),
 (192, 0.9313644170761108),
 (167, 0.8994714617729187),
 (221, 0.8780192136764526)]

In [ ]:
tagged_data[23]

TaggedDocument(words=['김성민', '실밸', '레이더', '선', '구매', '후', '결제', '서비스', '업체', '라르', '로이터', '연합뉴스', '세계', '경기', '악화', '유니콘', '기업', '가치', '조원', '이상', '비상', '스타트업', '기업가', '치가', '폭', '하락', '코로나', '팬데믹', '기간', '글로벌', '자금', '과도', '몸값', '제자리', '과정', '분석', '월스트리트저널', '지난', '현지', '시각', '스웨덴', '후불', '결제', '서비스', '업체', '라르', '달러', '조억원', '기업', '가치', '신규', '투자', '보도', '작년', '월', '투자', '때', '매', '기업', '가치', '달러', '조억원', '수준', '전', '세계', '명', '사용', '라르', '코로나', '팬데믹', '기간', '폭풍', '라르', '기업가', '달러', '산정', '투자', '유치', '글로벌', '경기', '침체', '시작', '펀딩', '마무리', '자존심', '차례', '기업가', '뒤', '달러', '신규', '투자자', '기업가', '치가', '투자자', '입장', '금액', '더', '확보', '수', '상장', '핀', '테크', '유니콘', '스트라이프', '기업', '가치', '폭락', '투자', '사', '피', '델리', '티', '최근', '보유', '스트라이프', '가치', '기존', '지난', '월', '해도', '스트라이프', '비상', '주식', '달러', '평가', '월', '말', '이보', '달러', '것', '장보기', '대행', '업체', '인스타', '카드', '지난', '월', '기업가', '기존', '달러', '달러', '자진', '삭감', '투자', '업계', '관계자', '투자', '위해', '그동안', '크게', '몸값', '다운', '그레이드', '현상', '업계', '전반', '확대', '가능성']

In [ ]:
return_docs = model.docvecs.most_similar(66,topn=3)
return_docs

/usr/local/lib/python3.7/dist-packages/gensim/matutils.py:737: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if np.issubdtype(vec.dtype, np.int):


[(30, 0.9380097985267639),
 (148, 0.8447320461273193),
 (191, 0.8400555849075317)]